# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/CRADIOv4/blob/main/using-cradio-in-fiftyone.ipynb)

# Using C-RADIO with your FiftyOne dataset

First, download a dataset. Let's use the `quickstart`:

In [ ]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/INQUIRE_Rerank",
    max_samples=1000
    )

Next, you register the zoo model source:


In [ ]:
import fiftyone.zoo as foz

foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/CRADIOv4"
    )

Finally, instantiate the model. Let's start with computing embeddings.

Note: Refer to the [README](https://github.com/harpreetsahota204/NVLabs_CRADIOV3/blob/main/README.md) for available model checkpoints.

In [ ]:
radio_embeddings_model = foz.load_zoo_model(
    "nv_labs/c-radio_v4-h",
    feature_format="NCHW", # you can also pass NLC here
)

You can compute embeddings as follows:

In [ ]:
dataset.compute_embeddings(
    model=radio_embeddings_model,
    embeddings_field="radio_embeddings",
)

Once you have your embeddings, you can compute the visualization to visualize in the FiftyOne App:

In [ ]:
import fiftyone.brain as fob

results = fob.compute_visualization(
    dataset,
    method="umap",  # "umap", "tsne", "pca", etc
    brain_key="radio_viz",
    embeddings="radio_embeddings"
)

You can also build a similarity index over the embeddings to find similar samples in your dataset:

In [ ]:
import fiftyone.brain as fob

results = fob.compute_similarity(
    dataset,
    backend="sklearn",  # "sklearn", "qdrant", "redis", etc
    brain_key="radio_sim",
    embeddings="radio_embeddings"
)

With your computed embeddings you can also perform other embeddings based workflows such as computing uniqueness values:

In [ ]:
import fiftyone.brain as fob

fob.compute_uniqueness(
    dataset,
    uniqueness_field="radio_uniqueness",
    similarity_index="radio_sim"
    )

You can also compute representativeness scores:

In [ ]:
import fiftyone.brain as fob

fob.compute_representativeness(
    dataset,
    representativeness_field="radio_representativeness",
    similarity_index="radio_sim"
    )

### Computing Spatial Features

You can also compute spatial features. To use this feature you need to set `output_type="spatial"`, additionally spatial features only supports `feature_format="NCHW"`.

You can choose to do some Gaussian smoothing if you'd like, just set `apply_smoothing=True` and choose a value for `smoothing_sigma`.

In [ ]:
radio_spatial_model = foz.load_zoo_model(
    "nv_labs/c-radio_v4-h",
    output_type="spatial",
    apply_smoothing=True, # if you want smoothing
    smoothing_sigma=0.51, # how much smoothing you want to apply
    feature_format="NCHW" #this is the required for the heatmap
) 

Notice that we are using the `apply_model` method here, as we are not computing 1D embeddings like above.

In [ ]:
dataset.apply_model(
    radio_spatial_model,
    "radio_spatial_features"
)

You can view your results in the app like so:

In [ ]:
fo.launch_app(dataset)

# ![CRADIO FiftyOne Demo](cradio-fiftyone.gif)